In [1]:
import pandas as pd

In [2]:
name_target = 'NU_NOTA_MT'
name_id = 'NU_INSCRICAO'

## Analisa os dados de teste e de treino
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print(f'Train shape: {df_train.shape}, Test shape: {df_test.shape}')

if name_target not in df_train.columns :
    print('ERROR: dataframe de treino não possui target')
    
elif df_test.columns.size >= (df_train.drop(columns=name_target).columns.size)\
        and all(elem in df_test for elem in df_train.drop(columns=name_target).columns) :
    print('ALERT: dataframes de teste e treino ok')

elif all(df_train[df_test.columns].columns == df_test.columns) :
    print('DONE: convertendo colunas dataframe de treino igual ao de teste + target')
    df_train = pd.concat([df_train[df_test.columns], df_train[name_target]], axis=1)
    print('DONE: gravando dataframe de treino formatado')
    df_train.to_csv('../data/train.csv', index=False)
else :
    print('ERROR: colunas dos Data frames de treino e testes são diferentes')

print(df_train.info())
del df_test, df_train

## Análise de dados nulos e zerados
df_train = pd.read_csv('../data/train.csv')
print(f'dados Nulos: {df_train[(df_train["TP_PRESENCA_LC"] == 0) | (df_train["TP_PRESENCA_LC"] == 2) ][name_target].unique()}')
print(f'dados Zerados: {df_train[(df_train["NU_NOTA_LC"] == 0)][name_target].unique()}')
del df_train

In [3]:
from data_source import DataSource
lin_null = [('TP_PRESENCA_LC', 0), ('TP_PRESENCA_LC', 2)]
lin_zero = [('NU_NOTA_LC', 0)]
data = DataSource(name_id=name_id, name_target=name_target, rows_remove=(lin_null + lin_zero))

## Análise de Feature
df_train = pd.read_csv('../data/train.csv')
import seaborn as sns
import statsmodels.api as sm
from IPython.core.pylabtools import figsize
%matplotlib inline
figsize(12, 8)
sns.set()
sns.heatmap(df_train.corr(), square=True);

var = df_train['TP_STATUS_REDACAO']
print(var.value_counts(dropna=False))
print(f'mean: {var.mean()}')
print(f'median: {var.median()}')
print(f'mode: {var.mode()}')
sns.distplot(var)

sns.scatterplot(var, df_train[name_target])

sns.lineplot(var, df_train[name_target])

sm.qqplot(var, fit=True, line="45");

In [4]:
## Optional manual analise columns
col_analise = [             #TYPE  ->[FILLNA  ENCODE ->[DROP_FIRST]]
    ('NU_INSCRICAO',        None), # only unique values
    ('CO_UF_RESIDENCIA',    None), # duplicated
    ('SG_UF_RESIDENCIA',    'cat',  None,   False),
    ('NU_IDADE',            'num',  None,   False),
    ('TP_SEXO',             'cat',  None,   False),
    #('TP_COR_RACA',         'cat'),
    ('TP_COR_RACA',         'cat',  None,   True,   True),
    ('TP_NACIONALIDADE',    None), # insignificant correlation
    ('TP_ST_CONCLUSAO',     'cat',  None,   False),
    ('TP_ANO_CONCLUIU',     None), # missing values
    #('TP_ESCOLA',           False),
    ('TP_ESCOLA',           'cat',  None,   True,   True),
    ('TP_ENSINO',           'cat',  None,   True,   False),
    ('IN_TREINEIRO',        'cat',  None,   False),	
    ('TP_DEPENDENCIA_ADM_ESC','cat',None,   True,   False),
    ('IN_BAIXA_VISAO',      None),	
    ('IN_CEGUEIRA',         None), 	
    ('IN_SURDEZ',           None), 	
    ('IN_DISLEXIA',         None), 	
    ('IN_DISCALCULIA',      None), 	
    ('IN_SABATISTA',        None), 	
    ('IN_GESTANTE',         None), 	
    ('IN_IDOSO',            None), 	
    ('TP_PRESENCA_CN',      None), 	
    ('TP_PRESENCA_CH',      None), 	
    ('TP_PRESENCA_LC',      None), 	
    ('CO_PROVA_CN',         None), 	
    ('CO_PROVA_CH',         None), 	
    ('CO_PROVA_LC',         None), 	
    ('CO_PROVA_MT',         None), 	
    ('NU_NOTA_CN',          'num',   459.8,  False),	
    ('NU_NOTA_CH',          'num',   532.0,  False),	
    ('NU_NOTA_LC',          'num',   None,   False),	
    #('NU_NOTA_REDACAO',     'num',   None,   False),	
    ('NU_NOTA_REDACAO',     None),	
    ('TP_LINGUA',           'cat',   None,   False),
    ('TP_STATUS_REDACAO',   None),	
    #('NU_NOTA_COMP1',       None),	
    ('NU_NOTA_COMP1',       'num',   None,   False),	
    #('NU_NOTA_COMP2',       None),	
    ('NU_NOTA_COMP2',       'num',   None,   False),
    #('NU_NOTA_COMP3',       None),	
    ('NU_NOTA_COMP3',       'num',   None,   False),
    #('NU_NOTA_COMP4',       None),
    ('NU_NOTA_COMP4',       'num',   None,   False),
    #('NU_NOTA_COMP5',       None),
    ('NU_NOTA_COMP5',       'num',   None,   False),
    ('Q001',                'cat',   None,   False),	
    ('Q002',                'cat',   None,   False),
    ('Q006',                'cat',   None,   False),
    ('Q024',                'cat',   None,   False),
    ('Q025',                'cat',   None,   False),
    #('Q026',                False),	
    ('Q026',                'cat',   None,   False),
    ('Q027',                None), # missing values	
    ('Q047',                'cat',   None,   False),
    ]

In [5]:
from preprocessing import Preprocessing
pre = Preprocessing(data, col_analise=col_analise)

#testes
df_new = pre.process()[0]
df_new.info()

In [6]:
#from experiments import Experiments
#Experiments(data).run_experiment()

In [7]:
from model_training import ModelTraining
model = ModelTraining(pre)

In [8]:
trained_model = model.training()

Training preprocessing
drop: NU_INSCRICAO
drop: CO_UF_RESIDENCIA
use: SG_UF_RESIDENCIA
use: NU_IDADE
use: TP_SEXO
use: TP_COR_RACA
	encode: [0 1 2 3 4 5]
	drop value: 0
drop: TP_NACIONALIDADE
use: TP_ST_CONCLUSAO
drop: TP_ANO_CONCLUIU
use: TP_ESCOLA
	encode: [1 2 3 4]
	drop value: 1
use: TP_ENSINO
	encode: [1. 2. 3.]
use: IN_TREINEIRO
use: TP_DEPENDENCIA_ADM_ESC
	encode: [1. 2. 3. 4.]
drop: IN_BAIXA_VISAO
drop: IN_CEGUEIRA
drop: IN_SURDEZ
drop: IN_DISLEXIA
drop: IN_DISCALCULIA
drop: IN_SABATISTA
drop: IN_GESTANTE
drop: IN_IDOSO
drop: TP_PRESENCA_CN
drop: TP_PRESENCA_CH
drop: TP_PRESENCA_LC
drop: CO_PROVA_CN
drop: CO_PROVA_CH
drop: CO_PROVA_LC
drop: CO_PROVA_MT
use: NU_NOTA_CN
	fill na with: 459.8
use: NU_NOTA_CH
	fill na with: 532.0
use: NU_NOTA_LC
drop: NU_NOTA_REDACAO
use: TP_LINGUA
drop: TP_STATUS_REDACAO
use: NU_NOTA_COMP1
use: NU_NOTA_COMP2
use: NU_NOTA_COMP3
use: NU_NOTA_COMP4
use: NU_NOTA_COMP5
use: Q001
use: Q002
use: Q006
use: Q024
use: Q025
use: Q026
drop: Q027
use: Q047
ALER

In [9]:
from model_inference import ModelInference
y_pred = ModelInference(trained_model).predict()

Preprocessing Data
drop: NU_INSCRICAO
drop: CO_UF_RESIDENCIA
use: SG_UF_RESIDENCIA
use: NU_IDADE
use: TP_SEXO
use: TP_COR_RACA
	encode: [0 1 2 3 4 5]
	drop value: 0
drop: TP_NACIONALIDADE
use: TP_ST_CONCLUSAO
drop: TP_ANO_CONCLUIU
use: TP_ESCOLA
	encode: [1 2 3]
	drop value: 1
use: TP_ENSINO
	encode: [1. 2. 3.]
use: IN_TREINEIRO
use: TP_DEPENDENCIA_ADM_ESC
	encode: [1. 2. 3. 4.]
drop: IN_BAIXA_VISAO
drop: IN_CEGUEIRA
drop: IN_SURDEZ
drop: IN_DISLEXIA
drop: IN_DISCALCULIA
drop: IN_SABATISTA
drop: IN_GESTANTE
drop: IN_IDOSO
drop: TP_PRESENCA_CN
drop: TP_PRESENCA_CH
drop: TP_PRESENCA_LC
drop: CO_PROVA_CN
drop: CO_PROVA_CH
drop: CO_PROVA_LC
drop: CO_PROVA_MT
use: NU_NOTA_CN
	fill na with: 459.8
use: NU_NOTA_CH
	fill na with: 532.0
use: NU_NOTA_LC
drop: NU_NOTA_REDACAO
use: TP_LINGUA
drop: TP_STATUS_REDACAO
use: NU_NOTA_COMP1
use: NU_NOTA_COMP2
use: NU_NOTA_COMP3
use: NU_NOTA_COMP4
use: NU_NOTA_COMP5
use: Q001
use: Q002
use: Q006
use: Q024
use: Q025
use: Q026
drop: Q027
use: Q047
ALERT: Num

In [10]:
df_answer = data.get_columns(name_columns=[name_id], is_train_stage=False)
df_answer[name_target] = y_pred.round(1)

df_removed = data.get_removed_rows(name_columns=[name_id], is_train_stage=False)
df_removed[name_target] = 0

df_answer = pd.concat([df_answer,df_removed])

In [11]:
df_answer.to_csv('../answer.csv', index=False)

In [ ]:
# histórico das submissões
# 92.12% StandardScaler->CatBoostEncoder->LinearRegression
# 93.24% StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.57% Seleção manual->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.58% Seleção manual(2)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.61% Seleção manual(3)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.60% Seleção manual(3)->Seleção RFE(50%)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento
# 93.52% Seleção manual(3)->Seleção RFE(12)->StandardScaler->CatBoostEncoder->LinearRegression->Pós Processamento